In [0]:
# create a notebook for LSTM prediction model 

# 1. Load the chosen dataset and try to see the vocab size 
# 2. Determine a MAX_VOCAB_SIZE incase you observe a vocab dict that is too large (choose the top/most frequent MAX_VOCAB_SIZE entries / Curse of dimensionality)
# 3. Determine a MAX_SEQUENCE LENGTH to vectorize for each review (Note: From splitter usually reviews will be limited to 400 + summary )

# DO NOT EDIT/ DELETE THIS BLOCK;



In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# set working directory -> you must set the path into which you have uploaded the zipped file
# this is required in the case of colab or local 
%cd /content/drive/My\ Drive/SNLP\ Project
# %cd Source/repos/Sentiment-Analysis-using-Deep-Learning

/content/drive/My Drive/SNLP Project


In [4]:
# list content of drive - verify you are where you are supposed to be
%ls

 CleantData_Apr-03-2020_01-31.zip          Lexical_analysis/
 Data_Balanced_100000_Apr-04-2020_06-54/   ModelResults/
 Data_Balanced_20000_Apr-03-2020_06-52/   'Project Ideas.gdoc'
 Data_Balanced_2000_Apr-03-2020_06-46/    'Project Proposal.gdoc'
 Electronics_5.json.gz                    'Report MetaData'/


In [5]:
# load your choice of dataset here . Specify paths as folder_datestring/file_datestring.zip
Train_ZipCSVFileName = 'Data_Balanced_20000_Apr-03-2020_06-52/Train_20000_Apr-03-2020_06-52.zip'
Test_ZipCSVFileName = 'Data_Balanced_20000_Apr-03-2020_06-52/Test_20000_Apr-03-2020_06-52.zip'

import pandas as pd
import numpy as np


df_train = pd.read_csv(Train_ZipCSVFileName)
df_train.info()

df_test = pd.read_csv(Test_ZipCSVFileName)
df_test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   overall         100000 non-null  float64
 1   reviewTime      100000 non-null  object 
 2   reviewerID      100000 non-null  object 
 3   asin            100000 non-null  object 
 4   reviewText      100000 non-null  object 
 5   summary         100000 non-null  object 
 6   unixReviewTime  100000 non-null  int64  
 7   reviewText_len  100000 non-null  int64  
 8   summary_len     100000 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 6.9+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   overall         10000 non-null  float64
 1   reviewTime      10000 non-null  object 
 2   reviewerID      10000 non-null 

In [6]:

# Get names of indexes for which column Age has value 30
index_neutrals_train = df_train[ df_train['overall'] == 3 ].index
index_neutrals_test = df_test[ df_test['overall'] == 3 ].index
 
# Delete these row indexes from dataFrame
df_train.drop(index_neutrals_train , inplace=True)
df_test.drop(index_neutrals_test , inplace=True)

df_train.loc[(df_train.overall == 1),'overall']= 1
df_train.loc[(df_train.overall == 2),'overall']= 1
df_train.loc[(df_train.overall == 4),'overall']= 5
df_train.loc[(df_train.overall == 5),'overall']= 5

df_test.loc[(df_test.overall == 1),'overall']= 1
df_test.loc[(df_test.overall == 2),'overall']= 1
df_test.loc[(df_test.overall == 4),'overall']= 5
df_test.loc[(df_test.overall == 5),'overall']= 5

df_train['reviewText_len'].describe()
# Since the mean average review size is around 145 chars and max is 400, I can safely set the max [summary + review] Text Limit to 400 
	

count    80000.000000
mean       148.534925
std        109.884928
min          1.000000
25%         49.000000
50%        129.000000
75%        228.000000
max        399.000000
Name: reviewText_len, dtype: float64

In [7]:
df_train['overall'].describe()
df_test['overall'].describe()

count    8000.000000
mean        3.000000
std         2.000125
min         1.000000
25%         1.000000
50%         3.000000
75%         5.000000
max         5.000000
Name: overall, dtype: float64

In [0]:
# The maximum number of words to be used. (most frequent)
MAX_VOCAB_SIZE = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 200

import uuid
folderGUID = uuid.uuid4().hex

# stupid shell way of converting variable to string 
!mkdir "ModelResults/$folderGUID"

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [16]:
tokenizer = Tokenizer(num_words= MAX_VOCAB_SIZE, filters='#$%&()*+<=>@[\\]^_`{|}~\t\n', lower=True)
tokenizer.fit_on_texts(df_train['reviewText'])
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 28112 unique tokens.


In [17]:
X = tokenizer.texts_to_sequences(df_train['reviewText'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)


Shape of data tensor: (80000, 250)


In [18]:
Y = pd.get_dummies(df_train['overall']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (80000, 2)


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Embedding(MAX_VOCAB_SIZE, EMBEDDING_DIM, input_length=X.shape[1]))
#model.add(SpatialDropout1D(0.2))
model.add(LSTM(200))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# run for small number of epochs then save 
epochs = 3

history = model.fit(X, Y, epochs=epochs)

Epoch 1/3
2500/2500 [==============================] - 1838s 735ms/step - loss: 0.2663 - accuracy: 0.8927
Epoch 2/3
2500/2500 [==============================] - 1850s 740ms/step - loss: 0.1641 - accuracy: 0.9394
Epoch 3/3
2500/2500 [==============================] - 1851s 740ms/step - loss: 0.1206 - accuracy: 0.9566


In [29]:
# Save the model to Local Disk 
# https://stackoverflow.com/questions/45424683/how-to-continue-training-for-a-saved-and-then-loaded-keras-model

filePath = "ModelResults/"+str(folderGUID)+"/model.h5"
model.save(filePath)
print("Saved model to disk : "+ str(folderGUID))



Saved model to disk : e9d6f7f0b19049f8a3567f51dfa2b019


In [27]:
# Run additional training if necessary & remember to resave it 

# Load the model
# model = load_model(filePath)

# Train more on the loaded model
history = model.fit(X, Y, epochs=epochs)

Epoch 1/3
2500/2500 [==============================] - 1834s 734ms/step - loss: 0.0398 - accuracy: 0.9872
Epoch 2/3
2500/2500 [==============================] - 1839s 735ms/step - loss: 0.0335 - accuracy: 0.9896
Epoch 3/3
2500/2500 [==============================] - 1830s 732ms/step - loss: 0.0281 - accuracy: 0.9910


In [28]:


X_test = tokenizer.texts_to_sequences(df_test['reviewText'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X_test.shape)

Y_Test = pd.get_dummies(df_test['overall']).values
print('Shape of label tensor:', Y_Test.shape)

accr = model.evaluate(X_test,Y_Test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Shape of data tensor: (8000, 250)
Shape of label tensor: (8000, 2)
250/250 [==============================] - 52s 207ms/step - loss: 0.5340 - accuracy: 0.9031
Test set
  Loss: 0.534
  Accuracy: 0.903


In [0]:
plt.title('Loss')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show();

plt.title('Accuracy')
plt.plot(history.history['acc'], label='train')
plt.plot(history.history['val_acc'], label='test')
plt.legend()
plt.show();


In [0]:
# Test for single review 
new_review = ['I am a victim of identity theft and someone stole my identity and personal information to open up a Visa credit card account with Bank of America. The following Bank of America Visa credit card account do not belong to me : XXXX.']
seq = tokenizer.texts_to_sequences(new_review)
padded = pad_sequences(seq, maxlen=MAX_SEQUENCE_LENGTH)
pred = model.predict(padded)


print(np.pred))
print(np.argmax(pred))